In [2]:
import numpy as np
import dicom
import glob
from matplotlib import pyplot as plt
import os
import cv2
import xgboost as xgb
import mxnet as mx
import pandas as pd
from sklearn.model_selection import train_test_split

In [17]:
def get_3d_data(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return np.stack([s.pixel_array for s in slices])

def get_data_id(path):
    sample_image = get_3d_data(path)
    print np.shape(sample_image)
    sample_image[sample_image == -2000] = 0
    # f, plots = plt.subplots(4, 5, sharex='col', sharey='row', figsize=(10, 8))

    batch = []
    cnt = 0
    dx = 40
    ds = 512
    for i in range(0, sample_image.shape[0] - 3, 3):
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (224, 224))
            tmp.append(img)

        tmp = np.array(tmp)
        batch.append(np.array(tmp))

        # if cnt < 20:
        #     plots[cnt // 5, cnt % 5].axis('off')
        #     plots[cnt // 5, cnt % 5].imshow(np.swapaxes(tmp, 0, 2))
        # cnt += 1

    # plt.show()
    batch = np.array(batch)
    print np.shape(batch)
    return batch

def get_extractor():
    model = mx.model.FeedForward.load('model/resnet-50', 0, ctx=mx.gpu(), numpy_batch_size=1)
    fea_symbol = model.symbol.get_internals()["flatten0_output"]
    feature_extractor = mx.model.FeedForward(ctx=mx.gpu(), symbol=fea_symbol, numpy_batch_size=16,
                                             arg_params=model.arg_params, aux_params=model.aux_params,
                                             allow_extra_params=True)

    return feature_extractor

path = '/home/zzz/Desktop/cogs181/sample/*'
BASE_PATH = '/home/zzz/Desktop/cogs181/npy_sample/'
def calc_features():
    net = get_extractor()
    for folder in glob.glob(path):
        batch = get_data_id(folder)
        feats = net.predict(batch)
#        print(feats.shape)
        np.save(os.path.join(BASE_PATH, folder), feats)
        print folder
calc_features()

/home/zzz/mxnet/python/mxnet/model.py:870: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
  **kwargs)
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:44: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.


OSError: [Errno 20] Not a directory: '/home/zzz/Desktop/cogs181/sample/0a38e7597ca26f9374f8ea2770ba870d.npy'

In [13]:
path = '/media/zzz/New/npy_data/'
sample = os.listdir(path)[1]
print sample

3043b5cc9733054f3ab5399c3c810406.npy


In [14]:
feature = np.load(path+sample)
print np.shape(feature)
sample2 = os.listdir(path)[100]
feature = np.load(path+sample2)
print np.shape(feature)

(85, 2048)
(43, 2048)


In [74]:
from sklearn.model_selection import GridSearchCV
def train_xgboost():
    x = []
    df = pd.read_csv('stage1_labels.csv')
#    print(df.head())
    x = np.array([np.mean(np.load('/media/zzz/New/pre_npy/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])
#        x1 = np.load('/media/zzz/New/pre_npy/%s.npy' % str(id))#.flatten]#.tolist()])

#    print np.shape(x)

    y = df['cancer'].as_matrix()

    trn_x, val_x, trn_y, val_y = train_test_split(x, y, random_state=42, stratify=y,
                                                                   test_size=0.20)
    xgb_model = xgb.XGBRegressor(learning_rate=0.05,
                           nthread=8,
                           subsample=0.80,
                           colsample_bytree=0.80,
                           seed=4242)
    clf = GridSearchCV(xgb_model,
                   {'max_depth': [2,4,6,8,10],
                    'n_estimators': [50,100,200],
                    'min_child_weight':[10,20]
                   }, verbose=1)

    clf.fit(trn_x, trn_y)#, verbose=1, eval_metric='auc', early_stopping_rounds=50)
    print(clf.best_score_)
    print(clf.best_params_)
    return clf

clf = train_xgboost()
# def make_submit():
#     clf = train_xgboost()

#     df = pd.read_csv('stage1_sample_submission.csv')

#    x = np.array([np.mean(np.load('/media/zzz/OS/pre_npy/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])

#    pred = clf.predict(x)

#    df['cancer'] = pred
#    df.to_csv('subm2.csv', index=False)
#    print(df.head())


if __name__ == '__main__':
    import time
    make_submit()

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed: 87.8min finished


-0.0166307140626
{'n_estimators': 50, 'max_depth': 2, 'min_child_weight': 20}
Fitting 3 folds for each of 36 candidates, totalling 108 fits


KeyboardInterrupt: 

In [14]:
folder = glob.glob('/media/zzz/OS/stage1/*')[0]
folder = 'a'
np.save(os.path.join(BASE_PATH, folder), '1')

In [100]:
def train_xgboost():
    x = []
    df = pd.read_csv('stage1_labels.csv')
#    print(df.head())
    x = np.array([np.mean(np.load('/media/zzz/New/Inception/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])
#        x1 = np.load('/media/zzz/New/pre_npy/%s.npy' % str(id))#.flatten]#.tolist()])

#    print np.shape(x)

    y = df['cancer'].as_matrix()

    trn_x, val_x, trn_y, val_y = train_test_split(x, y, random_state=42, stratify=y,
                                                                   test_size=0.20)
    xgb_model = xgb.XGBRegressor()

    clf = xgb.XGBRegressor(max_depth=10,
                           n_estimators=1500,
                           min_child_weight=10,
                           learning_rate=0.05,
                           nthread=8,
                           subsample=0.80,
                           colsample_bytree=0.80,
                           seed=4242)

    clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=1, eval_metric='error', early_stopping_rounds=50)
    return clf


def make_submit():
    clf = train_xgboost()

    df = pd.read_csv('stage1_sample_submission.csv')

    x = np.array([np.mean(np.load('/media/zzz/New/pre_npy/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])

    pred = clf.predict(x)

    df['cancer'] = pred
    df.to_csv('subm3.csv', index=False)
    print(df.head())


if __name__ == '__main__':
    import time
    make_submit()

[0]	validation_0-error:0.346429
Will train until validation_0-error hasn't improved in 50 rounds.
[1]	validation_0-error:0.339286
[2]	validation_0-error:0.303571
[3]	validation_0-error:0.303571
[4]	validation_0-error:0.285714
[5]	validation_0-error:0.289286
[6]	validation_0-error:0.271429
[7]	validation_0-error:0.282143
[8]	validation_0-error:0.267857
[9]	validation_0-error:0.271429
[10]	validation_0-error:0.275
[11]	validation_0-error:0.282143
[12]	validation_0-error:0.264286
[13]	validation_0-error:0.267857
[14]	validation_0-error:0.267857
[15]	validation_0-error:0.264286
[16]	validation_0-error:0.264286
[17]	validation_0-error:0.264286
[18]	validation_0-error:0.260714
[19]	validation_0-error:0.264286
[20]	validation_0-error:0.264286
[21]	validation_0-error:0.257143
[22]	validation_0-error:0.257143
[23]	validation_0-error:0.264286
[24]	validation_0-error:0.264286
[25]	validation_0-error:0.267857
[26]	validation_0-error:0.267857
[27]	validation_0-error:0.264286
[28]	validation_0-error